In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Load the dataset
df = pd.read_csv('NYPD_Shooting_Incident_Data__Historic_.csv')

# Initialize the Dash app
# Initialize the Dash app
app = dash.Dash(__name__)


# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("NYPD Shooting Incident Dashboard"),
    
    dcc.Dropdown(
        id='location-dropdown',
        options=[
            {'label': 'All Locations', 'value': 'All'},
        ] + [{'label': location, 'value': location} for location in df['BORO'].unique()],
        value='All',
        placeholder="Select a Location"
    ),

    html.Div(id='table-container'),

    dcc.Graph(id='perp-age-pie'),

    dcc.Graph(id='vic-age-pie'),

    dcc.Graph(id='perp-vic-bubble'),

    dcc.Graph(id='jurisdiction-donut'),

    dcc.Graph(id='gender-pie'),

    dcc.Graph(id='weekday-weekend-bar'),

    dcc.Graph(id='race-violin'),

    dcc.Graph(id='hourly-line'),

    dcc.Graph(id='race-gender-stacked-bar'),

    dcc.Graph(id='day-hour-heatmap'),

    dcc.Graph(id='treemap'),

    dcc.Graph(id='incident-map'),

    dcc.Graph(id='steamgraph')
])

# Define callback to update the table based on dropdown selection
@app.callback(
    Output('table-container', 'children'),
    [Input('location-dropdown', 'value')]
)
def update_table(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in filtered_df.columns])),
        html.Tbody([
            html.Tr([
                html.Td(filtered_df.iloc[i][col]) for col in filtered_df.columns
            ]) for i in range(min(len(filtered_df), 10))  # Display only first 10 rows
        ])
    ])
    return table

# Define callback to update perpetrator age pie chart
@app.callback(
    Output('perp-age-pie', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_perp_age_pie(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.pie(filtered_df, names='PERP_AGE_GROUP', title='Perpetrator Age Group Distribution',
                 color_discrete_sequence=px.colors.qualitative.Prism)
    return fig

# Define callback to update victim age pie chart
@app.callback(
    Output('vic-age-pie', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_vic_age_pie(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.pie(filtered_df, names='VIC_AGE_GROUP', title='Victim Age Group Distribution',
                 color_discrete_sequence=px.colors.qualitative.Prism)
    return fig

# Define callback to update perpetrator-victim bubble chart
@app.callback(
    Output('perp-vic-bubble', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_perp_vic_bubble(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location].copy()  # Create a copy of the DataFrame

    # Create a pivot table to count incidents for each combination of perpetrator age group and victim age group
    bubble_data = filtered_df.groupby(['PERP_AGE_GROUP', 'VIC_AGE_GROUP']).size().reset_index(name='Count')

    # Define the size of the bubble based on the count of incidents
    bubble_data['Count'] = bubble_data['Count'] * 5

    # Plot the bubble chart
    fig = px.scatter(bubble_data, x='PERP_AGE_GROUP', y='VIC_AGE_GROUP', size='Count', 
                     title='Perpetrator Age vs. Victim Age', size_max=30,
                     color_discrete_sequence=px.colors.qualitative.Prism)

    return fig

# Define callback to update jurisdiction donut chart
@app.callback(
    Output('jurisdiction-donut', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_jurisdiction_donut(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.pie(filtered_df, names='JURISDICTION_CODE', title='Jurisdiction Code Distribution',
                 color_discrete_sequence=px.colors.qualitative.Prism)
    return fig

# Define callback to update gender pie chart
@app.callback(
    Output('gender-pie', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_gender_pie(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.pie(filtered_df, names='VIC_SEX', title='Gender Distribution of Victims', 
                 color_discrete_sequence=px.colors.qualitative.Prism)
    return fig

# Define callback to update weekday-weekend bar chart
@app.callback(
    Output('weekday-weekend-bar', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_weekday_weekend_bar(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location].copy()  # Create a copy of the DataFrame

    # Extract weekday or weekend from OCCUR_DATE
    filtered_df['DayOfWeek'] = pd.to_datetime(filtered_df['OCCUR_DATE']).dt.dayofweek
    filtered_df['DayType'] = filtered_df['DayOfWeek'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

    # Count the number of incidents for each day type
    daytype_counts = filtered_df['DayType'].value_counts()

    # Plot the bar chart
    fig = px.bar(x=daytype_counts.index, y=daytype_counts.values, 
                 labels={'x': 'Day Type', 'y': 'Number of Incidents'}, 
                 title='Number of Incidents on Weekdays vs. Weekends', 
                 color_discrete_sequence=px.colors.qualitative.Prism)

    return fig

# Define callback to update race violin chart
@app.callback(
    Output('race-violin', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_race_violin(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.violin(filtered_df, x='PERP_RACE', y='VIC_RACE', 
                    title='Perpetrator Race vs. Victim Race',
                    color_discrete_sequence=px.colors.qualitative.Prism)
    return fig

# Define callback to update hourly line chart
@app.callback(
    Output('hourly-line', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_hourly_line(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location].copy()  # Create a copy of the DataFrame

    # Extract hour from OCCUR_TIME
    filtered_df['Hour'] = pd.to_datetime(filtered_df['OCCUR_TIME']).dt.hour

    # Count the number of incidents for each hour
    hourly_counts = filtered_df['Hour'].value_counts().sort_index()

    # Plot the line chart
    fig = px.line(hourly_counts, x=hourly_counts.index, y=hourly_counts.values, 
                  labels={'x': 'Hour of Day', 'y': 'Number of Incidents'}, 
                  title='Number of Incidents by Hour of Day', 
                  color_discrete_sequence=px.colors.qualitative.Prism)

    return fig

# Define callback to update race-gender stacked bar chart
@app.callback(
    Output('race-gender-stacked-bar', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_race_gender_stacked_bar(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location].copy()  # Create a copy of the DataFrame

    # Create a pivot table to count incidents for each combination of race and gender
    stacked_bar_data = filtered_df.groupby(['VIC_RACE', 'VIC_SEX']).size().unstack(fill_value=0)

    # Plot the stacked bar chart
    fig = go.Figure(data=[
        go.Bar(name='Male', x=stacked_bar_data.index, y=stacked_bar_data['M']),
        go.Bar(name='Female', x=stacked_bar_data.index, y=stacked_bar_data['F'])
    ])
    fig.update_layout(barmode='stack', title='Race-Gender Distribution of Victims', 
                      colorway=px.colors.qualitative.Prism)
    return fig

# Define callback to update day-hour heatmap
@app.callback(
    Output('day-hour-heatmap', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_day_hour_heatmap(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location].copy()  # Create a copy of the DataFrame

    # Extract day of week and hour from OCCUR_DATE and OCCUR_TIME
    filtered_df['DayOfWeek'] = pd.to_datetime(filtered_df['OCCUR_DATE']).dt.dayofweek
    filtered_df['Hour'] = pd.to_datetime(filtered_df['OCCUR_TIME']).dt.hour

    # Count the number of incidents for each combination of day of week and hour
    heatmap_data = filtered_df.groupby(['DayOfWeek', 'Hour']).size().unstack(fill_value=0)

    # Plot the heatmap with annotations
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        hoverongaps=False,
        colorscale='Viridis'))

    fig.update_layout(title='Number of Incidents by Day of Week and Hour',
                      xaxis_title='Hour of Day',
                      yaxis_title='Day of Week')

    return fig

# Define callback to update treemap
@app.callback(
    Output('treemap', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_treemap(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]

    # Replace NaN values in the DataFrame with 'Unknown' for the treemap visualization
    filtered_df['BORO'].fillna('Unknown', inplace=True)
    filtered_df['PERP_SEX'].fillna('Unknown', inplace=True)
    filtered_df['VIC_SEX'].fillna('Unknown', inplace=True)

    # Create a treemap
    fig = px.treemap(filtered_df, path=['BORO', 'PERP_SEX', 'VIC_SEX'], 
                     values='STATISTICAL_MURDER_FLAG', 
                     color='PERP_SEX', 
                     color_discrete_map={'M': 'darkblue', 'F': 'darkred', 'U': 'darkgreen', 'Unknown': 'grey'})
    return fig

# Define callback to update incident map
@app.callback(
    Output('incident-map', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_incident_map(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.scatter_mapbox(filtered_df, lat='Latitude', lon='Longitude', color='STATISTICAL_MURDER_FLAG', 
                            hover_data=['PERP_SEX', 'VIC_SEX'], zoom=10,
                            color_discrete_map={True: 'red', False: 'blue'})
    fig.update_layout(mapbox_style="carto-positron", title='Incident Map')
    return fig

# Define callback to update steamgraph
@app.callback(
    Output('steamgraph', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_steamgraph(selected_location):
    if selected_location == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['BORO'] == selected_location]
    fig = px.parallel_categories(filtered_df, dimensions=['VIC_RACE', 'PERP_SEX', 'BORO'],
                                  color='STATISTICAL_MURDER_FLAG', 
                                  title='Steamgraph of Victim Race, Perpetrator Sex, and Borough', 
                                  color_continuous_scale=px.colors.sequential.Aggrnyl)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\vishw\AppData\Local\Temp\ipykernel_25548\119989191.py:212: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\vishw\AppData\Local\Temp\ipykernel_25548\119989191.py:261: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

